In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from simulation import dropna
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go
import pandas

In [2]:
master = pandas.read_csv('./fort_longer.123',sep='\t')

In [3]:
master['dX'] = master['x'].diff()
master['dY'] = master['y'].diff()
master['dZ'] = master['z'].diff()

In [4]:
trace = go.Scatter3d(x=master['dX'],y=master['dY'],z=master['dZ'],mode='lines')
py.iplot([trace])

#### synchrony

$\phi = x_M - x_S - D \\$
$D = a_0 + a_1 sin(\omega t)$

In [5]:
a0,a1,w = 3.5,2.5,1*pi
epsilon = 1.5
delTime = .1

def zetaInvar(stateA,stateB,t):
    return -asarray([w*a1*cos(w*t)]*len(stateA))
def chi(stateA,stateB,t):
    return stateA-stateB-a0-a1*sin(w*t)
def zetaNull(stateA,stateB,t):
    return asarray([0,0,0],dtype=float)

## slave evolution

In [6]:
def slaveEvolution(master,zeta):
    slave = zeros((len(master),3))
    stateB = master[['x','y','z']].iloc[0].to_numpy()
    #slave[0] = stateB
    for index,master_state in master.iloc[1:].iterrows():
        time = master_state['t']
        stateB = slave[index-1]
        stateA = master_state[['x','y','z']].to_numpy()

        dA = master_state[['dX','dY','dZ']].to_numpy()
        coupling = zeta(stateA,stateB,time) 
        coupling += chi(stateA,stateB,time)*epsilon
        stateB += delTime*(dA+coupling)
        slave[index] = stateB
    slave = pandas.DataFrame(slave,columns=['x','y','z'])
    return slave

In [7]:
slave_zeta = slaveEvolution(master,zetaInvar)
slave_null = slaveEvolution(master,zetaNull)
D = a0-a1*sin(w*master['t'])
slave_truth = master[['x','y','z']] - expand_dims(D,axis=1)

In [8]:
master_trace = go.Scatter3d(name='master',x=master['x'],y=master['y'],z=master['z'],mode='lines')
slave_zeta_trace = go.Scatter3d(name='slave_zeta',x=slave_zeta['x'],y=slave_zeta['y'],z=slave_zeta['z'],mode='lines')
slave_null_trace = go.Scatter3d(name='slave_null',x=slave_null['x'],y=slave_null['y'],z=slave_null['z'],mode='lines')
slave_truth_trace = go.Scatter3d(name='true',x=slave_truth['x'],y=slave_truth['y'],z=slave_truth['z'],mode='lines')
py.iplot([master_trace,slave_zeta_trace,slave_null_trace,slave_truth_trace])

## synchrony error

In [9]:
dimension = 'y'
py.iplot([go.Scatter(name='truth_'+dimension,x=master['t'],y=slave_truth[dimension]),
         go.Scatter(name='zeta_'+dimension,x=master['t'],y=slave_zeta[dimension].shift(0)),
         go.Scatter(name='null_'+dimension,x=master['t'],y=slave_null[dimension].shift(0))])

In [10]:
error_zeta = simulation.syncError(slave_truth[['x','y','z']].to_numpy(),slave_zeta[['x','y','z']].shift(-5).to_numpy())
error_null = simulation.syncError(slave_truth[['x','y','z']].to_numpy(),slave_null[['x','y','z']].shift(-5).to_numpy())

In [11]:
py.iplot([go.Scatter(name='x',x=master['t'],y=error_zeta[:,0],mode='lines'),
         go.Scatter(name='y',x=master['t'],y=error_zeta[:,1],mode='lines'),
         go.Scatter(name='z',x=master['t'],y=error_zeta[:,2],mode='lines')])

In [12]:
py.iplot([go.Scatter(name='x',x=master['t'],y=error_null[:,0],mode='lines'),
         go.Scatter(name='y',x=master['t'],y=error_null[:,1],mode='lines'),
         go.Scatter(name='z',x=master['t'],y=error_null[:,2],mode='lines')])

In [13]:
print('error zeta:',dropna(error_zeta.sum(axis=1)).mean())
print('error null:',dropna(error_null.sum(axis=1)).mean())

error zeta: 12.021474233365266
error null: 17.939853070490308


In [14]:
print(simulation.lagMinimaSyncError(slave_truth,slave_zeta,10))
print(simulation.lagMinimaSyncError(slave_truth,slave_null,10))

(11.063521546927282, -6)
(17.894981001779726, -5)


In [15]:
error_zeta_trace = go.Scatter(name='zeta',x=master['t'],y=error_zeta.sum(axis=1))
error_null_trace = go.Scatter(name='null',x=master['t'],y=error_null.sum(axis=1))
py.iplot([error_zeta_trace,error_null_trace])

--------------------